# 작업 1유형

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')

### 1
마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0\~9 : 0 , 10\~19 : 10)

In [10]:
df.head()

df['age2'] = (df.age//10*10)
print(df.age2.value_counts().index[0])

30


### 2
마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?

In [13]:
print(df.age2.value_counts().values[0])

5056


### 3
나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?

In [16]:
print(df[(df.age >= 25)&(df.age < 29)&(df.housing=='yes')].ID.count())

504


### 4
numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?

In [47]:
# df.info()  # int : 'ID', 'age', 'balance', 'day', 'campaign', 'pdays', 'previous', 'age2'
df4 = df.drop(columns=['ID', 'age', 'balance', 'day', 'campaign', 'pdays', 'previous', 'age2'])
df4_ = df4.nunique().sort_values(ascending=False).to_frame('cnt').reset_index()

print(df4_[df4_['cnt'] == df4_.cnt.max()].iloc[:,0])

0      job
1    month
Name: index, dtype: object


### 5
balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?

In [49]:
result = df[df.balance >= df.balance.mean()].sort_values('ID', ascending=False).head(100)
print(result.balance.mean())

3473.73


### 6
가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)

In [58]:
print(df.groupby(['day', 'month']).count().ID.sort_values(ascending=False).index[0])

(15, 'may')


### 7
데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라

In [63]:
from scipy.stats import shapiro
s, p = shapiro(df[df.job == 'unknown'].age)
print(p)

0.19612720608711243


### 8
age와 balance의 상관계수를 구하여라

In [74]:
print(df[['age', 'balance']].corr()['balance'][0])

0.10198734763981462


### 9
y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라

In [77]:
from scipy.stats import chi2_contingency

cdf = pd.crosstab(df.y, df.education)
print(chi2_contingency(cdf)[1])

7.901201277473551e-29


### 10
각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?

In [92]:
t = df.groupby(['job', 'marital']).size().reset_index()
pivotdf = t.pivot_table(index='job', columns='marital')[0]
pivotdf = pivotdf.fillna(0)
pivotdf['ratio'] = pivotdf['divorced']/pivotdf['married']

result = pivotdf.sort_values('ratio').ratio.values[-1]
print(result)

0.2831050228310502


# 작업 2유형


In [93]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

display(train.head())
display(test.head())
display(submission.head())

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


In [99]:
# print(train.isna().sum(), test.isna().sum())  # 결측치 없음
# print(train.info())
# print(test.info())
## object : 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', y(target)
y = train.y
# category_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# print(train[category_cols].nunique())
# print(test[category_cols].nunique())
# 너무 많은 카테고리 job, month 제거

## drop
train_d = train.drop(columns=['ID', 'job', 'month', 'y'])
test_d = test.drop(columns=['ID', 'job', 'month'])
## encoded
train_dd = pd.get_dummies(train_d)
test_dd = pd.get_dummies(test_d)
test_dd = test_dd[train_dd.columns]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(train_dd, y, test_size=0.3, random_state=0, stratify=y)
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

val_pred = rf.predict(X_val)
val_proba = rf.predict_proba(X_val)[:,1]
print('roc score:', roc_auc_score(y_val, val_proba))

predict = rf.predict(test_dd)
pd.DataFrame({'ID':test.ID, 'predict':predict}).to_csv('results/exam1.csv', index=False)

roc score: 0.7410710746297771
